In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as linalg
import scipy.sparse as sparse
import scipy.sparse.linalg as linalgs
from matplotlib import animation
from IPython.display import HTML

In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 100.

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(( -0.1, 1.1))
ax.set_ylim((- 0.3, 0.3))

line, = ax.plot([], [], lw=2)

nframes = 500

def init():
    line.set_data([], [])
    return (line,)

## String

In [ ]:
class StringProb():
    """Class for setting up a string wave eigenvector problem"""
    def p(self, x=None):
        """Return the elasticity at x"""
        return(1.) 
    def rho(self, x=None):
        """Return density at x"""
        return(1.) 
    def q(self, x=None):
        """Return springiness at x"""
        return(0.)
    def __init__(self, N=201, **kwargs):
        self.kwargs = kwargs
        self.N = N
        self.h = 1. / (np.float64(N) + 1.)
        self.x = (np.arange(self.N, dtype=np.float64) + 1.) * self.h
        diagonal = ((self.p(self.x - 0.5 * self.h) +
                     self.p(self.x + 0.5 * self.h)) / self.h**2 
                    + self.q(self.x)) / self.rho(self.x)
        lower_diagonal = - (self.p(self.x[1:] - 0.5 * self.h)) / self.h**2 / self.rho(self.x[1:]) 
        upper_diagonal = - (self.p(self.x[1:] - 0.5 * self.h)) / self.h**2 / self.rho(self.x[0:-1]) 
        self.A = sparse.diags(diagonals=[lower_diagonal, diagonal, upper_diagonal],
                              offsets=[-1, 0, 1], shape=(N, N))
    def eig(self):
        # Note that the sparse version of "eig" failed here; I know this problem should 
        # yield sines and cosines. A lesson to check your package's performance! So I 
        # work on the full matrix.
        vals, vecs = linalg.eig(self.A.toarray())
        isort = np.argsort(vals)
        self.evals = vals[isort]
        self.evecs = vecs[:, isort]
    def set_coeffs(self, a=None, b=None):
        self.a = a
        self.b = b
    def fit_coeffs(self, d=None):
        """Fit coefficients for eigenvectors for a set of displacements"""
        (u, w, vt) = linalg.svd(self.evecs)
        inz = np.where(w > 1.e-15)[0]
        invw = np.zeros(len(w))
        invw[inz] = 1. / w[inz] 
        einv = vt.transpose().dot(np.diag(invw)).dot(u.transpose())
        return(einv.dot(d)) 
    def frame(self, i):
        period =  2. * np.pi / np.sqrt(self.evals[0])
        t = np.float64(i) / np.float64(nframes) * period * 4
        x = self.x
        coeffs = (self.a * np.cos(np.sqrt(self.evals) * t) +
                  self.b * np.sin(np.sqrt(self.evals) * t))
        y = self.evecs.dot(coeffs).flatten()
        line.set_data(x, y)
        return (line,)


In [ ]:
st = StringProb()
st.eig()
print(st.A.toarray())

In [ ]:
print(st.evals[0:30])

In [ ]:
print(np.sqrt(st.evals[0:30] / st.evals[0]))

In [ ]:
for i in np.arange(3):
    plt.plot(st.x, st.evecs[:, i])

In [ ]:
plt.plot(st.x, st.evecs[:, 10])

In [ ]:
plt.plot(st.x, st.evecs[:, 80])
plt.xlim(0., 0.1)

In [ ]:
acoeffs = np.zeros(len(st.evals))
acoeffs[0] = 1.
acoeffs[1] = 0.5
acoeffs[2] = 0.25

bcoeffs = np.zeros(len(st.evals))

st.set_coeffs(a=acoeffs, b=bcoeffs)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, st.frame, init_func=init,
                               frames=nframes, interval=40, blit=True)

HTML(anim.to_jshtml())

In [ ]:
initial = (- (st.x - 0.5)**4 + 0.5**4) * 4.
coeffs = st.fit_coeffs(d=initial)

plt.plot(st.x, initial)
plt.plot(st.x, st.evecs.dot(coeffs))

In [ ]:
plt.plot(coeffs)
plt.xlim((-1., 10))

In [ ]:
st.set_coeffs(a=coeffs, b=bcoeffs)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, st.frame, init_func=init,
                               frames=nframes, interval=40, blit=True)

HTML(anim.to_jshtml())

## String which is a bit springy

In [ ]:
class StringProbSpringy(StringProb):
    def q(self, x):
        return(self.kwargs['q0'])

In [ ]:
stq = StringProbSpringy(q0=40.1)
stq.eig()

In [ ]:
print(stq.A.toarray())

Eigenvalues are higher, which is just saying there is a minimum frequency that is higher, like the dispersion related says must be the case.

In [ ]:
print(stq.evals[0:10])

In [ ]:
print(np.sqrt(stq.evals[0:10] / stq.evals[0]))

But the eigenfunctions are the same for the patterns. Note that the way they evolve will be different, because, as just shown above, their frequencies are no longer in a harmonic sequence. 

In [ ]:
for i in np.arange(3):
    plt.plot(stq.x, stq.evecs[:, i])

In [ ]:
initial = (- (stq.x - 0.5)**4 + 0.5**4) * 3.
coeffs = stq.fit_coeffs(d=initial)

plt.plot(stq.x, initial)
plt.plot(stq.x, stq.evecs.dot(coeffs))

In [ ]:
print(coeffs[0:10])

In [ ]:
stq.set_coeffs(a=coeffs, b=bcoeffs)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, stq.frame, init_func=init,
                               frames=nframes, interval=40, blit=True,
                               repeat_delay=2000)

HTML(anim.to_jshtml())

## String which is heavy in the middle

In [ ]:
class StringProbDensity(StringProb):
    def rho(self, x):
        rrho = np.ones(len(x))
        rrho[(x > 0.40) & (x < 0.60)] = 1. + self.kwargs['rhocen']
        return(rrho)

In [ ]:
std = StringProbDensity(rhocen=20., N=1001)
std.eig()

In [ ]:
print(std.A.toarray())

In [ ]:
print(std.evals[0:10])

In [ ]:
plt.plot(std.x, std.rho(std.x))

In [ ]:
for i in np.arange(4):
    plt.plot(std.x, std.evecs[:, i])

In [ ]:
plt.plot(std.x, std.evecs[:, 10])

In [ ]:
initial = (- (std.x - 0.5)**4 + 0.5**4) * 3.
coeffs = std.fit_coeffs(d=initial)

plt.plot(std.x, initial)
plt.plot(std.x, std.evecs.dot(coeffs))

In [ ]:
bcoeffs = np.zeros(std.N)

std.set_coeffs(a=coeffs, b=bcoeffs)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, std.frame, init_func=init,
                               frames=nframes, interval=40, blit=True,
                               repeat_delay=2000)

HTML(anim.to_jshtml())

In [ ]:
initial = np.zeros(std.N)
half = np.arange(std.N // 2)
initial[half] = np.sin(np.pi * std.x[half] / 0.5)**2 * 0.25
coeffs = std.fit_coeffs(d=initial)

plt.plot(std.x, initial)
plt.plot(std.x, std.evecs.dot(coeffs))

In [ ]:
bcoeffs = np.zeros(std.N)

std.set_coeffs(a=coeffs, b=bcoeffs)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, std.frame, init_func=init,
                               frames=nframes, interval=40, blit=True,
                               repeat_delay=2000)

HTML(anim.to_jshtml())

In [ ]:
plt.plot(coeffs)
plt.xlim((0., 100.))